Для выполнения заданий возьмите данные из папки data о продажах. Данные распределены по 4 папкам:


*  sales_train.csv - данные о продажах с января 2013 по октябрь 2015.

  *   date - дата продажы
  *   date_block_num - номер месяца по порядку следования
  *   shop_id - идентификатор магазина
  *   item_id - идентификатор товара
  *  item_cnt_day - количество проданного товара одного вида в день
*   items.csv - подробная информация о товарах
  *   item_id - идентификатор товара
  *   item_name - название товара
  *   category_id - идентификатор категории товара
*   item_categories.csv - подробная информация о категориях товаров
  *   item_category_id - идентификатор категории товара
  *   category_name - название категории товара
*   shops.csv - подробная информация о магазинах
  * shop_id - идентификатор магазина
  * shop_name - название магазина


# Задание 1


1.   Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2.   Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).


In [ ]:
sales_train <- read.csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1jSJU4h7A-TNKuWH0ClzYx3dfCJiTD21g')
item_categories <- read.csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1oIUxVxRLj03vVHShVCay8ZHW9a9D9M0m')
items <- read.csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1QdYmeqN_oq_fcbe77SX7xb_hCviNd0HO')
shops <- read.csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1gRFn6w4uRFoUmbZrjaspLjJdtRCMzxyx')
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
#1.1
result = sales_train %>%
           group_by(item_id) %>%
           summarise(total_sales = sum(item_cnt_day))

result = result[order(result$total_sales, decreasing = TRUE), ]
result = head(result, n=10)
result

item_id,total_sales
<int>,<dbl>
20949,187642
2808,17245
3732,16642
17717,15830
5822,14515
3734,11688
6675,10289
3731,10099
1855,10032


In [ ]:
#1.2
sales_copy <- sales_train

rule36 <- filter(sales_copy, date_block_num >= 24 & date_block_num < 36)
rule24 <- filter(sales_copy, date_block_num >= 12 & date_block_num < 24)
rule12 <- filter(sales_copy, date_block_num < 12)

result[(result$item_id %in% rule12$item_id) & (result$item_id %in% rule24$item_id) & (result$item_id %in% rule36$item_id), ]

item_id,total_sales
<int>,<dbl>
20949,187642
2808,17245
3732,16642
17717,15830
5822,14515
3734,11688
6675,10289
1855,10032
16787,9227


# Задание 2

1.   Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2.   Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.



In [ ]:
items_copy <- items
sales_newcopy <- sales_train

items_frame <- merge(sales_newcopy, items_copy, by='item_id')
items_frame = items_frame %>%
                  group_by(category_id) %>%
                  summarise(total_sales = sum(item_cnt_day))

max_cat <- items_frame[items_frame$total_sales == max(items_frame$total_sales), ]
min_cat <- items_frame[items_frame$total_sales == min(items_frame$total_sales), ]
#items_frame[which.max(items_frame$total_sales), ]

print('Минимальные категории')
item_categories[item_categories$item_category_id %in% min_cat$category_id, ]
print('Максимальная категория')
item_categories[item_categories$item_category_id %in% max_cat$category_id, ]

[1] "Минимальные категории"


,item_category_name,item_category_id
,<chr>,<int>
11,Game consoles - PS2,10
52,Books - Cognitive literature,51


[1] "Максимальная категория"


,item_category_name,item_category_id
,<chr>,<int>
41,Cinema - DVD,40


# Задание 3

1.   Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2.   Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.



In [ ]:
#3.1
shop_sales <- sales_train
sales_frame <- sales_train
items_newcopy <- items

shop_sales <- shop_sales %>%
              group_by(shop_id) %>%
              summarise(total_sales = sum(item_cnt_day))
shop_sales <- head(shop_sales[order(shop_sales$total_sales, decreasing = TRUE), ], 5)

sales_frame <- merge(sales_frame, items_newcopy, by = 'item_id')

min_frame <- sales_frame[(sales_frame$category_id %in% min_cat$category_id), ]

min_frame <- min_frame[min_frame$shop_id %in% shop_sales$shop_id, ]

min_frame <- min_frame %>%
                group_by(item_id, shop_id) %>%
                summarise(total_sales = sum(item_cnt_day))



min_frame <- min_frame %>%
                group_by(item_id) %>%
                summarise(mean_sales = mean(total_sales))

min_frame
#продаж для минимальной категории нет в топовых магазинах

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.


item_id,mean_sales
<int>,<dbl>


In [ ]:
max_frame <- sales_frame[(sales_frame$category_id %in% max_cat$category_id), ]

max_frame <- max_frame[max_frame$shop_id %in% shop_sales$shop_id, ]

max_frame <- max_frame %>%
                group_by(item_id, shop_id) %>%
                summarise(total_sales = sum(item_cnt_day))



max_frame <- max_frame %>%
                group_by(item_id) %>%
                summarise(mean_sales = mean(total_sales))

max_frame

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.


item_id,mean_sales
<int>,<dbl>
0,1.000000
2,2.000000
3,2.000000
4,1.000000
5,1.000000
6,1.000000
7,1.000000
8,2.000000
9,1.000000


In [ ]:
#3.2
sales_frame <- sales_train
items_newnewcopy <- items

sales_newframe <- merge(sales_frame, items_newnewcopy, by = 'item_id')

max_months <- sales_newframe[(sales_newframe$item_id %in% max_frame$item_id) | (sales_newframe$item_id %in% min_frame$item_id), ]

max_months <- max_months %>%
                group_by(item_id, date_block_num) %>%
                summarise(total_sales = sum(item_cnt_day))

min_months <- max_months[order(max_months$item_id, max_months$total_sales, decreasing = FALSE), ]
min_months <- distinct(min_months, item_id, .keep_all = TRUE)

max_months <- distinct(max_months, item_id, .keep_all = TRUE)
max_months

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.


item_id,date_block_num,total_sales
<int>,<int>,<dbl>
0,20,1
2,19,1
3,18,1
4,20,1
5,23,1
6,18,1
7,23,1
8,19,1
9,19,1


In [ ]:
min_months

item_id,date_block_num,total_sales
<int>,<int>,<dbl>
0,20,1
2,19,1
3,18,1
4,20,1
5,23,1
6,18,1
7,23,1
8,19,1
9,19,1


# Задание 4

1.   Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2.   В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.



In [ ]:
#4.1 
jeneral_category <- max_frame %>%
                      full_join(min_frame)
sales_newframe <- sales_newframe[sales_newframe$item_id %in% jeneral_category$item_id, ]

sales_mean <- sales_newframe %>%
                group_by(date_block_num, category_id) %>%
                summarise(mean_price = mean(item_price))

sales_mean

Joining, by = c("item_id", "mean_sales")
`summarise()` has grouped output by 'date_block_num'. You can override using
the `.groups` argument.


date_block_num,category_id,mean_price
<int>,<int>,<dbl>
0,40,244.8589
1,40,252.3997
2,40,260.8980
3,40,254.9732
4,40,247.7220
5,40,245.9637
6,40,236.8638
7,40,234.4875
8,40,241.8383


In [ ]:
sales_median <- sales_newframe %>%
                group_by(date_block_num, category_id) %>%
                summarise(median_price = median(item_price))

sales_median

`summarise()` has grouped output by 'date_block_num'. You can override using
the `.groups` argument.


date_block_num,category_id,median_price
<int>,<int>,<dbl>
0,40,199
1,40,203
2,40,231
3,40,199
4,40,199
5,40,199
6,40,149
7,40,149
8,40,198


In [ ]:
year2013 <- sales_mean[(sales_mean$date_block_num < 12), ]

for (i in 2 : (nrow(year2013) - 1))
{
  if((year2013[i, 'mean_price'] < year2013[i-1, 'mean_price']) & (year2013[i, 'mean_price'] < year2013[i+1, 'mean_price']))
  {
    print(year2013[i, ])
  }
}

# A tibble: 1 × 3
# Groups:   date_block_num [1]
  date_block_num category_id mean_price
           <int>       <int>      <dbl>
1              7          40       234.


In [ ]:
year2014 <- sales_mean[(sales_mean$date_block_num < 24) & (sales_mean$date_block_num >= 12), ]

for (i in 2 : (nrow(year2014) - 1))
{
  if((year2014[i, 'mean_price'] < year2014[i-1, 'mean_price']) & (year2014[i, 'mean_price'] < year2014[i+1, 'mean_price']))
  {
    print(year2014[i, ])
  }
}

# A tibble: 1 × 3
# Groups:   date_block_num [1]
  date_block_num category_id mean_price
           <int>       <int>      <dbl>
1             16          40       254.
# A tibble: 1 × 3
# Groups:   date_block_num [1]
  date_block_num category_id mean_price
           <int>       <int>      <dbl>
1             21          40       264.


In [ ]:
year2015 <- sales_mean[(sales_mean$date_block_num < 36) & (sales_mean$date_block_num >= 24), ]

for (i in 2 : (nrow(year2015) - 1))
{
  if((year2015[i, 'mean_price'] < year2015[i-1, 'mean_price']) & (year2015[i, 'mean_price'] < year2015[i+1, 'mean_price']))
  {
    print(year2015[i, ])
  }
}

# A tibble: 1 × 3
# Groups:   date_block_num [1]
  date_block_num category_id mean_price
           <int>       <int>      <dbl>
1             25          40       256.
# A tibble: 1 × 3
# Groups:   date_block_num [1]
  date_block_num category_id mean_price
           <int>       <int>      <dbl>
1             31          40       273.
